In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.datasets import fetch_openml
from keras.datasets import mnist


import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
import PIL.Image
from matplotlib import patches


2025-10-21 13:07:52.136424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761077272.151545   76066 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761077272.156232   76066 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761077272.168339   76066 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761077272.168357   76066 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761077272.168359   76066 computation_placer.cc:177] computation placer alr

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Data Access

In [3]:
data_dir = Path("..","data")
models_dir = Path("..","models")

(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## Defining Bench Mark Function

In [4]:
import time

def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        print(f"---- Epoch {epoch_num} ----")
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)

        print(f"Execution time for epoch {epoch_num} : {time.perf_counter() - start_time}")
    print("Total Execution time:", time.perf_counter() - start_time)

* So single epoch took 2857.236867081 seconds so ~47 mins mamjority of that time went to data generation since we spent only 0.01s perbatch for "training"

## Data Generation With Tensor Flow

In [25]:
ALL_MNIST_DATA_PIXELS_TF = tf.constant(x_train, dtype=tf.float32)
ALL_MNIST_DATA_CLASSES_TF = tf.constant(y_train, dtype=tf.float32)

# number of digits to overlay on canvas
num_of_digits = tf.constant(3, dtype=tf.int32)

# max digits to define the shape of prediction output
MAX_DIGITS = tf.constant(5, dtype=tf.int32)


# Sample Base Digits
@tf.function
def get_sample_indices(dataset, size=5):
    dataset_len = tf.shape(dataset)[0] - 1
    random_indices = tf.random.uniform(
        shape=[size], minval=0, maxval=dataset_len, dtype=tf.int32)
    return random_indices


@tf.function
def sample_base_digits(num_of_digits):
    """
    Sample a specified number of digit images and their class labels from the master MNIST dataset.

    Args:
        num_of_digits (int): Number of digit samples to retrieve.

    Returns:
        tuple: (sample_pixels, sample_values)
            sample_pixels (np.ndarray): Array of digit images with shape (num_of_digits, 28, 28, 1).
            sample_values (np.ndarray): Array of class labels with shape (num_of_digits, 1).
    """
    sample_indices = get_sample_indices(
        ALL_MNIST_DATA_CLASSES_TF, size=num_of_digits)
    sample_pixels = tf.gather(ALL_MNIST_DATA_PIXELS_TF,
                              indices=sample_indices, axis=0, batch_dims=1)
    sample_pixels = tf.reshape(sample_pixels, shape=(num_of_digits, 28, 28))

    sample_values = tf.gather(
        ALL_MNIST_DATA_CLASSES_TF, indices=sample_indices, axis=0, batch_dims=1)
    sample_values = tf.reshape(sample_values, shape=(num_of_digits, 1))
    return sample_pixels, sample_values


augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomTranslation(
        height_factor=0.2, width_factor=0.2, fill_value=0.0, fill_mode="constant", seed=42),

    tf.keras.layers.RandomZoom(
        height_factor=0.2, width_factor=0.2, fill_value=0.0, fill_mode="constant", seed=42),

    tf.keras.layers.RandomRotation(
        factor=0.1, fill_value=0.0, fill_mode="constant", seed=42),
])


def augment_digits(digits, debug=False):
    """
    Apply random augmentations (translation, zoom, rotation) to a batch of digit images.

    Args:
        digits (np.ndarray): Array of digit images to augment.
        debug (bool, optional): If True, displays before/after images for each digit. Defaults to False.

    Returns:
        np.ndarray: Augmented digit images as a numpy array.
    """
    # step 2: apply random augmentation
    augmented_tensor_digits = augmentation(digits)
    return augmented_tensor_digits


@tf.function
def get_min_max(active_rows, active_cols):
    # find x_min, x_max
    # step 1 find indices for active x
    non_zero_active_cols = tf.where(active_cols != 0)
    # get the first and last active x as x_min and x_max
    x_min = tf.cast(tf.reduce_min(non_zero_active_cols[:, 1]), dtype=tf.int32)
    x_max = tf.cast(tf.reduce_max(non_zero_active_cols[:, 1]), dtype=tf.int32)

    ##
    non_zero_active_rows = tf.where(active_rows != 0)
    y_min = tf.cast(tf.reduce_min(non_zero_active_rows[:, 0]), dtype=tf.int32)
    y_max = tf.cast(tf.reduce_max(non_zero_active_rows[:, 0]), dtype=tf.int32)

    return x_min, x_max, y_min, y_max


def create_blank_canvas(shape=(100, 100, 1)):
    """
    Create a blank canvas for placing digit images.

    Args:
        shape (tuple, optional): Shape of the canvas. Defaults to (100, 100, 1).

    Returns:
        np.ndarray: Blank canvas array.
    """
    canvas = tf.zeros(shape=(100, 100, 1), dtype=tf.float32)
    return canvas


def create_prediction_object():
    """
    Create an empty prediction object for storing digit detection results.

    Returns:
        np.ndarray: Prediction array of shape (MAX_DIGITS, 15).
    """
    prediction = tf.zeros(shape=(MAX_DIGITS, 15), dtype=tf.float32)
    return prediction


@tf.function
def calculate_tight_bbox(pixels, class_values, padding=1):
    """Creates bounding box for the digits in pixel tensor and returns a concatenated tensor with bounding box and class

    Args:
        pixels (_type_): (m,28,28) tensor of pixels
        class_values (_type_): (m,1) tensor of class values
    """
    # step 1: calculate active rows and cols
    # active rows will have shape of (m,28) where each 28 pixels represent sum of 28 pixels in the col
    active_rows = tf.reduce_sum(pixels, axis=2)
    tf.print("----- active_rows shape : ", tf.shape(active_rows))
    # active rows will have shape of (m,28) where each 28 pixels represent sum of 28 pixels in the row
    active_cols = tf.reduce_sum(pixels, axis=1)
    tf.print("----- active_cols shape : ", tf.shape(active_cols))

    # step 2: find non zero coordinates
    # create boolean mask for active rows
    non_zero_row_mask = active_rows > 0
    # TIL - tf.print by default converts boolean to integer values 1, 0
    tf.print("----- non_zero_row_mask shape : ", tf.shape(non_zero_row_mask))
    # find coordinates using the boolean mask
    non_zero_row_coordinates = tf.where(non_zero_row_mask)
    tf.print("----- non_zero_row_coordinates shape : ",
             tf.shape(non_zero_row_coordinates))
    # find min and max row coordinates using segment_min, segment_max
    segment_data = non_zero_row_coordinates[:, 1]
    segment_ids = non_zero_row_coordinates[:, 0]
    # FYI - since we are parsing thru the rows it gives us y-coordinates of the image
    y_min = tf.math.segment_min(data=segment_data, segment_ids=segment_ids)
    y_max = tf.math.segment_max(data=segment_data, segment_ids=segment_ids)
    tf.print("----- y_min shape : ", tf.shape(y_min))
    tf.print("----- y_max shape : ", tf.shape(y_max))

    # create boolean mask for active cols
    non_zero_col_mask = active_cols > 0
    # TIL - tf.print by default converts boolean to integer values 1, 0
    tf.print("----- non_zero_col_mask shape : ", tf.shape(non_zero_col_mask))
    # find coordinates using the boolean mask
    non_zero_col_coordinates = tf.where(non_zero_col_mask)
    tf.print("----- non_zero_col_coordinates shape : ",
             tf.shape(non_zero_col_coordinates))
    # find min and max row coordinates using segment_min, segment_max
    segment_data = non_zero_col_coordinates[:, 1]
    segment_ids = non_zero_col_coordinates[:, 0]
    # FYI - since we are parsing thru the rows it gives us y-coordinates of the image
    x_min = tf.math.segment_min(data=segment_data, segment_ids=segment_ids)
    x_max = tf.math.segment_max(data=segment_data, segment_ids=segment_ids)
    tf.print("----- x_min shape : ", tf.shape(x_min))
    tf.print("----- x_max shape : ", tf.shape(x_max))

    # step 3: add padding to pixels
    # calculate padding condition for x_min
    x_min_padding_cond = x_min > 0
    padded_x_min = x_min - padding
    x_min = tf.where(x_min_padding_cond, padded_x_min, x_min)

    # calculate padding condition for x_max
    x_max_padding_cond = x_max < 27
    padded_x_max = x_max + padding
    x_max = tf.where(x_max_padding_cond, padded_x_max, x_max)

    # calculate padding condition for y_min
    y_min_padding_cond = y_min > 0
    padded_y_min = y_min - padding
    y_min = tf.where(y_min_padding_cond, padded_y_min, y_min)

    # calculate padding condition for y_max
    y_max_padding_cond = y_max < 27
    padded_y_max = y_max + padding
    y_max = tf.where(y_max_padding_cond, padded_y_max, y_max)

    # step 4: calculate x_center & y_center
    x_center = tf.round((x_min + x_max) / 2)
    y_center = tf.round((y_min + y_max) / 2)
    tf.print("----- x_center shape : ", tf.shape(x_center))
    tf.print("----- x_center shape : ", tf.shape(x_center))

    # step 5: calculate width and height
    width = x_max - x_min + 1
    height = y_max - y_min + 1
    tf.print("----- width shape : ", tf.shape(width))
    tf.print("----- width shape : ", tf.shape(width))

    # reshape all the values to match class_values
    x_min = tf.reshape(x_min, shape=(-1, 1))
    x_max = tf.reshape(x_max, shape=(-1, 1))
    y_min = tf.reshape(y_min, shape=(-1, 1))
    y_max = tf.reshape(y_max, shape=(-1, 1))
    x_center = tf.reshape(x_center, shape=(-1, 1))
    y_center = tf.reshape(y_center, shape=(-1, 1))
    width = tf.reshape(width, shape=(-1, 1))
    height = tf.reshape(height, shape=(-1, 1))

    tf.print("----- x_min shape : ", tf.shape(x_min))
    tf.print("----- x_max shape : ", tf.shape(x_max))
    tf.print("----- y_min shape : ", tf.shape(y_min))
    tf.print("----- y_max shape : ", tf.shape(y_max))
    tf.print("----- x_center shape : ", tf.shape(x_center))
    tf.print("----- y_center shape : ", tf.shape(y_center))
    tf.print("----- width shape : ", tf.shape(width))
    tf.print("----- height shape : ", tf.shape(height))

    # casting all values to same dtype
    x_min = tf.cast(x_min, dtype=tf.int32)
    x_max = tf.cast(x_max, dtype=tf.int32)
    y_min = tf.cast(y_min, dtype=tf.int32)
    y_max = tf.cast(y_max, dtype=tf.int32)
    x_center = tf.cast(x_center, dtype=tf.int32)
    y_center = tf.cast(y_center, dtype=tf.int32)
    width = tf.cast(width, dtype=tf.int32)
    height = tf.cast(height, dtype=tf.int32)
    class_values = tf.cast(class_values, dtype=tf.int32)

    bounding_box = tf.concat(
        [x_min, x_max, y_min, y_max, x_center, y_center, width, height, class_values], axis=-1)
    tf.print("----- bounding_box shape : ", tf.shape(bounding_box))
    return bounding_box


augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomTranslation(
        height_factor=0.2, width_factor=0.2, fill_value=0.0, fill_mode="constant", seed=42),

    tf.keras.layers.RandomZoom(
        height_factor=0.2, width_factor=0.2, fill_value=0.0, fill_mode="constant", seed=42),

    tf.keras.layers.RandomRotation(
        factor=0.1, fill_value=0.0, fill_mode="constant", seed=42),
])


@tf.function
def augment_digits(pixels):

    augmented_pixels = augmentation(pixels)
    return augmented_pixels


def generate_training_example_tf(x, y, debug=True):
    """
    Generate a training example by placing digits on a canvas and creating the corresponding prediction object.

    Args:
        x (np.ndarray): Input digit image(s).
        y (np.ndarray): Corresponding class label(s).

    Returns:
        tuple: (canvas, prediction) where canvas is the composed image and prediction is the label array.
    """
    pixels = tf.reshape(x, shape=(-1, 28, 28))
    class_values = tf.reshape(y, shape=(-1, 1))

    if debug:
        tf.print("----- pixels shape : ", tf.shape(pixels))
        tf.print("----- class_values shape : ", tf.shape(class_values))
    # step 1: sample additional digits
    if num_of_digits - 1 > 0:
        additional_digits, additional_class_values = sample_base_digits(
            num_of_digits - 1)
        pixels = tf.concat([pixels, additional_digits], axis=0)
        class_values = tf.concat(
            [class_values, additional_class_values], axis=0)

        if debug:
            tf.print("----- pixels with additional_digits shape : ",
                     tf.shape(pixels))
            tf.print(
                "----- class_values with additional_class_values shape : ", tf.shape(class_values))

    # step 2: augment digits
    augmented_pixels = augment_digits(pixels)
    if debug:
        tf.print("----- augmented_pixels shape : ", tf.shape(augmented_pixels))

    # step 3: calculate bounding box
    class_values_with_bbox = calculate_tight_bbox(
        augmented_pixels, class_values)

    # # step 4: create blank canvas and prediction
    canvas = create_blank_canvas()
    prediction = create_prediction_object()

    # # step 5: place digit on canvas
    # canvas, class_bbox = place_digit_on_canvas(
    #     canvas, pixels, class_with_bbox, debug=debug)

    # # step 6: translate bbox to prediction object
    # prediction = translate_bbox_to_prediction(
    #     class_bbox, prediction, debug=debug)

    # # print(f"Final canvas shape {canvas.shape}, final prediction shape {prediction.shape}")
    tf.print("----- Processed Sample -----")
    return x, y

In [26]:
## temporary just selecting first 32 records to test quickly
X_tensor = tf.convert_to_tensor(x_train[:32], dtype=tf.float32)
y_tensor = tf.convert_to_tensor(y_train[:32], dtype=tf.float32)
batch_size = 32

print(tf.shape(X_tensor))
print(tf.shape(y_tensor))

raw_dataset = tf.data.Dataset.from_tensor_slices((X_tensor, y_tensor))

# Use the wrapper inside the map
tf_processed_dataset = raw_dataset.map(generate_training_example_tf).batch(batch_size=batch_size)

tf.Tensor([32 28 28], shape=(3,), dtype=int32)
tf.Tensor([32], shape=(1,), dtype=int32)


### Bench Marking

In [27]:
benchmark(tf_processed_dataset.prefetch(tf.data.AUTOTUNE), num_epochs=1)

---- Epoch 0 ----
----- pixels shape :  [1 28 28]
----- class_values shape :  [1 1]
----- pixels with additional_digits shape :  [3 28 28]
----- class_values with additional_class_values shape :  [3 1]
----- augmented_pixels shape :  [3 28 28]
----- active_rows shape :  [3 28]
----- active_cols shape :  [3 28]
----- non_zero_row_mask shape :  [3 28]
----- non_zero_row_coordinates shape :  [44 2]
----- y_min shape :  [3]
----- y_max shape :  [3]
----- non_zero_col_mask shape :  [3 28]
----- non_zero_col_coordinates shape :  [62 2]
----- x_min shape :  [3]
----- x_max shape :  [3]
----- x_center shape :  [3]
----- x_center shape :  [3]
----- width shape :  [3]
----- width shape :  [3]
----- x_min shape :  [3 1]
----- x_max shape :  [3 1]
----- y_min shape :  [3 1]
----- y_max shape :  [3 1]
----- x_center shape :  [3 1]
----- y_center shape :  [3 1]
----- width shape :  [3 1]
----- height shape :  [3 1]
----- bounding_box shape :  [3 9]
----- Processed Sample -----
----- pixels shape :  